### Table of Content

- [Table of Content](#table-of-content)



## Purpose — Data Processing & Harmonisation
This notebook prepares the meta-data for our systematic review and meta-analysis. Goals:
- Reproducible import & cleaning of the data from the extraction Excel file
- Harmonisation of heterogeneous fields
- Parsing of semi-structured values into analysis-ready columns
- Keying rows by 'study_id'
- Structuring arms in long format for easier downstream analysis.

## Set Working directory

In [1]:
setwd("/Users/stevenschepanski/Documents/04_ANALYSIS/CWEMetaAnalysis/")

## Install and Load Required Packages in R
[Back to Table of Content](#table-of-content)

In [2]:
# install.packages(c("tidyverse", "readxl", "janitor", "writexl", "lubridate", "stringi"))

library(tidyverse)
library(readxl)
library(janitor)
library(writexl)
library(lubridate)
library(stringi)
library(haven)

Warning message:
“package ‘ggplot2’ was built under R version 4.3.3”
Warning message:
“package ‘tibble’ was built under R version 4.3.3”
Warning message:
“package ‘purrr’ was built under R version 4.3.3”
Warning message:
“package ‘lubridate’ was built under R version 4.3.3”
── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.2     ✔ tibble    3.3.0
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.0.4     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Warning message:
“package ‘readxl’ was built under R version 4.3.3”
Warning message:
“package ‘janitor’ was built under R version 4.3.3”

Attaching package: ‘janitor’


The following objects are masked

## Load in excel file
[Back to Table of Content](#table-of-content)

In [3]:
FILE_PATH <- "data/CWE_data_extraction_finalHARMONISED.xlsx"  # path to your master extraction file
SHEET     <- 1

# Output directory for processed files
OUT_DIR <- "processed"
if (!dir.exists(OUT_DIR)) dir.create(OUT_DIR, recursive = TRUE)

In [4]:
# Sheet map
sheet_map <- c(
  metadata     = "Sheet2_Metadaten",
  mental       = "Sheet4_MentalHealth",
  nonmental    = "Sheet5_NonPsych"
)

In [5]:
# Sanity check: make sure all sheets exist
available <- readxl::excel_sheets(FILE_PATH)
missing   <- setdiff(unname(sheet_map), available)
if (length(missing) > 0) {
  stop("These sheets were not found in the workbook: ",
       paste(missing, collapse = ", "), call. = FALSE)
}

In [6]:
# Helper to read + clean a sheet
read_clean <- function(path, sheet_name) {
  readxl::read_excel(path, sheet = sheet_name, guess_max = 10000) %>%
    janitor::clean_names()
}

In [7]:
# Read all required sheets
metadata     <- read_clean(FILE_PATH, sheet_map["metadata"])
mental       <- read_clean(FILE_PATH, sheet_map["mental"])
nonmental    <- read_clean(FILE_PATH, sheet_map["nonmental"])

## Inspect and harmonise metadata
[Back to Table of Content](#table-of-content)

In [8]:
## Basic structure
glimpse(metadata)

Rows: 12
Columns: 32
$ study_id                             <dbl> 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11…
$ author                               <chr> "Blades et al.", "Buijze et al.",…
$ year_of_publication                  <dbl> 2024, 2016, 2023, 2025, 2022, 201…
$ doi                                  <chr> "https://doi.org/10.1016/j.cpnec.…
$ country                              <chr> "USA", "Netherlands", "Switzerlan…
$ study_design                         <chr> "RCT", "RCT", "RCT", "RCT", "Pre-…
$ language                             <chr> "English", "English", "English", …
$ funding_information                  <chr> "grant from John W. Brick Mental …
$ sampling_procedure                   <chr> "convenience sample; local commun…
$ multiple_intervention_groups         <chr> "no", "yes (3)", "no", "no", "no"…
$ total_sample_size                    <dbl> 84, 3018, 42, 30, 48, 156, 36, 82…
$ n_itt_intervention                   <dbl> 43, 2300, 22, 15, 48, 156, 25, 49…
$ n_itt_control    

In [9]:
## Categorical variables

metadata <- metadata %>%
  mutate(
    # study_id: keep numeric, add a factor version
    study_id_factor = factor(study_id),

    author                   = factor(author),
    year_of_publication      = factor(year_of_publication),
    country                  = factor(country),
    study_design             = factor(study_design),
    language                 = factor(language),
    sampling_procedure       = factor(sampling_procedure),
    multiple_intervention_groups = factor(multiple_intervention_groups)
  )

In [10]:
## Numeric variables

num_from_char <- function(x) {
  if (is.numeric(x)) return(x)

  x_clean <- trimws(as.character(x))
  x_clean[x_clean %in% c("na", "NA", "n/a", "nr", "NR", "")] <- NA_character_
  as.numeric(x_clean)
}

metadata <- metadata %>%
  mutate(
    total_sample_size          = num_from_char(total_sample_size),
    n_itt_intervention         = num_from_char(n_itt_intervention),
    n_itt_control              = num_from_char(n_itt_control),
    n_pp_intervention          = num_from_char(n_pp_intervention),
    n_pp_control               = num_from_char(n_pp_control),
    mean_age_itt_intervention  = num_from_char(mean_age_itt_intervention),
    mean_age_itt_control       = num_from_char(mean_age_itt_control),
    median_age_pp_intervention = num_from_char(median_age_pp_intervention),
    median_age_pp_control      = num_from_char(median_age_pp_control),
    mean_age_pp_intervention   = num_from_char(mean_age_pp_intervention),
    mean_age_pp_control        = num_from_char(mean_age_pp_control)
  )

In [11]:
## Age ranges -> lower / upper bounds

split_age_range <- function(x) {
  # returns a tibble with lower, upper (numeric)
  x_chr <- tolower(trimws(as.character(x)))
  x_chr[x_chr %in% c("na", "n/a", "nr", "")] <- NA_character_

  # pattern: first and second integer in the string, e.g. "30-60 (eligibility)"
  m <- str_match(x_chr, "(\\d+)\\D+(\\d+)")
  tibble(
    lower = as.numeric(m[, 2]),
    upper = as.numeric(m[, 3])
  )
}

# ITT intervention
age_itt_int   <- split_age_range(metadata$age_range_itt_intervention)
# ITT control
age_itt_ctrl  <- split_age_range(metadata$age_range_itt_control)
# PP intervention
age_pp_int    <- split_age_range(metadata$age_range_pp_intervention)
# PP control
age_pp_ctrl   <- split_age_range(metadata$age_range_pp_control)

metadata <- metadata %>%
  mutate(
    age_itt_intervention_lower = age_itt_int$lower,
    age_itt_intervention_upper = age_itt_int$upper,
    age_itt_control_lower      = age_itt_ctrl$lower,
    age_itt_control_upper      = age_itt_ctrl$upper,
    age_pp_intervention_lower  = age_pp_int$lower,
    age_pp_intervention_upper  = age_pp_int$upper,
    age_pp_control_lower       = age_pp_ctrl$lower,
    age_pp_control_upper       = age_pp_ctrl$upper
  )

In [12]:
## Sex proportions -> female / male ratio

sex_prop_female <- function(x) {
  map_dbl(x, function(xx) {
    if (is.na(xx)) return(NA_real_)
    s <- tolower(trimws(as.character(xx)))
    if (s %in% c("na", "n/a", "nr", "")) return(NA_real_)

    # explicit counts "18F / 7M", "7F 9M", "5F 8M", "2F 4M", "16F 17M", ...
    has_f_counts <- str_detect(s, "(\\d+\\s*f\\b)|(\\d+f\\b)")
    has_m_counts <- str_detect(s, "(\\d+\\s*m\\b)|(\\d+m\\b)")

    if (has_f_counts && has_m_counts) {
      nums <- str_extract_all(s, "\\d+\\.?\\d*")[[1]]
      if (length(nums) >= 2) {
        n_f <- as.numeric(nums[1])
        n_m <- as.numeric(nums[2])
        tot <- n_f + n_m
        if (!is.na(tot) && tot > 0) {
          return(n_f / tot)
        }
      }
    }

    # percent reporting, e.g. "399, 56% female", "0, 0% female",
    # "only for completers: 91/121 female = 75.2%", 
    # "44.82 M vs 55.17% F"
    if (str_detect(s, "%")) {
      nums <- str_extract_all(s, "\\d+\\.?\\d*")[[1]]
      if (length(nums) >= 1) {
        p <- as.numeric(nums[length(nums)])  # assume last number is % female
        if (!is.na(p)) {
          if (p > 1) return(p / 100)
          return(p)
        }
      }
    }

    # pure proportion or single numeric, e.g. "0.752"
    nums <- str_extract_all(s, "\\d+\\.?\\d*")[[1]]
    if (length(nums) == 1) {
      p <- as.numeric(nums[1])
      if (!is.na(p)) {
        if (p > 1) return(p / 100)  # if written as percent
        return(p)                   # already in [0,1]
      }
    }

    # special case: "100% female" without being caught above
    if (str_detect(s, "100%") && str_detect(s, "female")) {
      return(1)
    }

    NA_real_
  })
}

metadata <- metadata %>%
  mutate(
    sex_itt_intervention_p_female = sex_prop_female(sex_proportion_itt_intervention),
    sex_itt_intervention_p_male   = if_else(
      is.na(sex_itt_intervention_p_female),
      NA_real_,
      1 - sex_itt_intervention_p_female
    ),

    sex_itt_control_p_female = sex_prop_female(sex_proportion_itt_control),
    sex_itt_control_p_male   = if_else(
      is.na(sex_itt_control_p_female),
      NA_real_,
      1 - sex_itt_control_p_female
    ),

    sex_pp_intervention_p_female = sex_prop_female(sex_proportion_pp_intervention),
    sex_pp_intervention_p_male   = if_else(
      is.na(sex_pp_intervention_p_female),
      NA_real_,
      1 - sex_pp_intervention_p_female
    ),

    sex_pp_control_p_female = sex_prop_female(sex_proportion_pp_control),
    sex_pp_control_p_male   = if_else(
      is.na(sex_pp_control_p_female),
      NA_real_,
      1 - sex_pp_control_p_female
    )
  )

In [13]:
## Health condition: Healthy / mental / other / na / nr

metadata <- metadata %>%
  mutate(
    health_condition_recoded = case_when(
      # missing / not reported
      str_to_lower(health_condition) %in% c("na", "n/a") ~ "na",
      str_to_lower(health_condition) %in% c("nr", "not reported") ~ "nr",

      # any reference to depression / anxiety / mental health
      str_detect(str_to_lower(health_condition),
                 "depress|anxiet|psychiatr|mental") ~ "mental health issue",

      # explicitly healthy
      str_detect(str_to_lower(health_condition), "healthy") ~ "healthy",

      # everything else = physical / other conditions
      TRUE ~ "other health condition"
    ) %>%
      factor(levels = c("healthy",
                        "mental health issue",
                        "other health condition",
                        "na",
                        "nr"))
  )

In [14]:
## Study_Design harmonisation
metadata <- metadata %>%
  mutate(
    study_design = case_when(
      study_id %in% 1:4   ~ "RCT",
      study_id %in% 5:6   ~ "Non-randomised CT",
      study_id %in% 7:8   ~ "Observational cohort",
      study_id %in% 9:12  ~ "Non-randomised CT",
      TRUE                ~ NA_character_
    ),
    study_design = factor(study_design,
                          levels = c("RCT", "Non-randomised CT", "Observational cohort"))
  )


In [15]:
## Drop unnecessary columns
metadata <- metadata %>%
  dplyr::select(
    study_id, study_id_factor,
    author, year_of_publication, country, language,
    study_design, sampling_procedure, funding_information, comments,

    total_sample_size,
    n_itt_intervention, n_itt_control,
    n_pp_intervention, n_pp_control,

    mean_age_itt_intervention, mean_age_itt_control,
    median_age_pp_intervention, median_age_pp_control,
    mean_age_pp_intervention, mean_age_pp_control,

    age_itt_intervention_lower, age_itt_intervention_upper,
    age_itt_control_lower, age_itt_control_upper,
    age_pp_intervention_lower, age_pp_intervention_upper,
    age_pp_control_lower, age_pp_control_upper,

    sex_itt_intervention_p_female, sex_itt_intervention_p_male,
    sex_itt_control_p_female, sex_itt_control_p_male,
    sex_pp_intervention_p_female, sex_pp_intervention_p_male,
    sex_pp_control_p_female, sex_pp_control_p_male,

    health_condition_recoded
  )


## Inspect and harmonise mental data
[Back to Table of Content](#table-of-content)

In [16]:
## Basic structure
glimpse(mental)

Rows: 175
Columns: 24
$ study_id          <dbl> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1…
$ instrument        <chr> "CES-D", "CES-D", "CES-D", "CES-D", "CES-D", "CES-D"…
$ instrument_domain <chr> "Depression", "Depression", "Depression", "Depressio…
$ subscale          <chr> "na", "na", "na", "na", "na", "na", "na", "na", "na"…
$ scale_range       <chr> "0-60", "0-60", "0-60", "0-60", "0-60", "0-60", "0-2…
$ timepoint_label   <chr> "Pre", "Pre", "Post", "Post", "Follow-Up", "Follow-u…
$ timepoint_days    <chr> "0.0", "0.0", "21.0", "21.0", "105.0", "105.0", "0.0…
$ group             <chr> "Intervention", "Control", "Intervention", "Control"…
$ analysis_set      <chr> "PP", "PP", "PP", "PP", "PP", "PP", "PP", "PP", "PP"…
$ n                 <dbl> 42, 41, 37, 41, 23, 23, 43, 41, 37, 41, 23, 23, 42, …
$ mean              <chr> "22.7", "24.2", "18.13", "17.6", "15.83", "16.09", "…
$ sd                <chr> "7.72", "9.72", "10.17", "10.76", "8.74", "11.66", "…
$ se              

In [17]:
## Helper
num_from_char <- function(x) {
  if (is.numeric(x)) return(x)
  x_clean <- trimws(as.character(x))
  x_clean[x_clean %in% c("na", "NA", "n/a", "nr", "NR", "")] <- NA_character_
  suppressWarnings(as.numeric(x_clean))
}

In [18]:
## Harmonise 'timepoint_label'
harm_timepoint_label <- function(x) {
  s <- str_to_lower(trimws(as.character(x)))

  case_when(
    # follow-up first (specific labels)
    str_detect(s, "follow") ~ "Follow-Up",

    # immediate post-immersion time points
    str_detect(s, "post-immersion") |
      str_detect(s, "min after immersion") ~ "Post-Immersion",

    # baseline / pre
    str_detect(s, "baseline") |
      str_detect(s, "autumn \\(pre\\)") |
      str_detect(s, "\\bpre\\b") |
      str_detect(s, "day 1 \\(prev week\\)") ~ "Pre",

    # everything else: post-baseline (post, day 5, weeks, 4 months, etc.)
    TRUE ~ "Post"
  )
}

In [19]:
## Harmonise 'timepoint_days'
tp_days_numeric <- function(x) {
  s <- str_to_lower(trimws(as.character(x)))
  out <- rep(NA_real_, length(s))

  # pure numbers: already days
  is_num <- str_detect(s, "^\\d+$")
  out[is_num] <- as.numeric(s[is_num])

  # baseline definitions
  out[str_detect(s, "baseline")] <- 0
  out[str_detect(s, "day 0")]   <- 0   # e.g. "Day 0 (baseline)"

  # months / weeks
  out[str_detect(s, "4 months")]         <- 4 * 30      # assumption: 30 days / month
  out[str_detect(s, "after 8 weeks")]    <- 8 * 7
  out[str_detect(s, "end of week\\s*1")] <- 1 * 7
  out[str_detect(s, "end of week\\s*2")] <- 2 * 7
  out[str_detect(s, "end of week\\s*3")] <- 3 * 7
  out[str_detect(s, "end of week\\s*4")] <- 4 * 7

  # 30- / 90-day follow-up (if as text)
  out[str_detect(s, "30-day")] <- 30
  out[str_detect(s, "90-day")] <- 90

  # immediate post-immersion points – fractions of a day
  out[str_detect(s, "within 15min")]          <- 15  / 60 / 24
  out[str_detect(s, "30min post-immersion")]  <- 30  / 60 / 24
  out[str_detect(s, "180min post-immersion")] <- 180 / 60 / 24
  out[str_detect(s, "14min after immersion")] <- 14  / 60 / 24

  # before immersion (negative fractional day; mid-point of 18.5–23.5 min)
  out[str_detect(s, "18\\.5min-23\\.5min before immersion")] <- -21 / 60 / 24

  # everything else stays NA unless covered above
  out
}

In [20]:
## Harmonise 'group'
harm_group <- function(x) {
  s <- str_trim(as.character(x))
  s_lower <- str_to_lower(s)

  case_when(
    str_detect(s_lower, "singlegroup") ~ "SingleGroup",

    str_detect(s_lower, "intervention\\+control") ~ "Intervention+Control",

    # pure control (no "+control" composite)
    str_detect(s_lower, "control") &
      !str_detect(s_lower, "intervention\\+control") ~ "Control",

    # all flavours of intervention, e.g. "Intervention(30s)" etc.
    str_detect(s_lower, "intervention") ~ "Intervention",

    TRUE ~ NA_character_
  )
}

In [21]:
## Mental health sheet
mental <- mental %>%
  mutate(
    ## IDs
    study_id        = as.integer(study_id),
    study_id_factor = factor(study_id),

    ## Instruments / domains
    instrument        = factor(instrument),
    instrument_domain = factor(instrument_domain),

    ## Subscale: standardise "n.r." -> "nr" once
    subscale = subscale %>%
      str_replace_all("n\\.r\\.", "nr") %>%
      factor(),

    ## Keep raw timepoint & group for later use
    timepoint_label_raw = as.character(timepoint_label),
    timepoint_days_raw  = as.character(timepoint_days),
    group_raw           = as.character(group),

    ## Original factor versions (for reference)
    timepoint_label = factor(timepoint_label),
    timepoint_days  = factor(timepoint_days),
    group           = factor(group),

    analysis_set = factor(analysis_set),

    ## Numeric outcome fields
    n           = num_from_char(n),
    mean        = num_from_char(mean),
    sd          = num_from_char(sd),
    se          = num_from_char(se),
    ci_l        = num_from_char(ci_l),
    ci_u        = num_from_char(ci_u),
    median      = num_from_char(median),
    iqr_l       = num_from_char(iqr_l),
    iqr_u       = num_from_char(iqr_u),
    change_mean = num_from_char(change_mean),
    change_sd   = num_from_char(change_sd)
  ) %>%

  ## Drop unneeded columns from the original sheet
  dplyr::select(
    -scale_range,
    -test_statistic,
    -p_value,
    -missing_data,
    -notes
  ) %>%

  ## Add harmonised variables (for analysis)
  mutate(
    timepoint_label_harmonised = factor(
      harm_timepoint_label(timepoint_label_raw),
      levels = c("Pre", "Post", "Post-Immersion", "Follow-Up")
    ),
    timepoint_days_num = tp_days_numeric(timepoint_days_raw),

    group_harmonised = factor(
      harm_group(group_raw),
      levels = c("Intervention", "Control", "Intervention+Control", "SingleGroup")
    )
  )

## Inspect and harmonise non mental data
[Back to Table of Content](#table-of-content)

In [22]:
## Basic structure
glimpse(nonmental)

Rows: 219
Columns: 25
$ study_id         <dbl> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,…
$ outcome_name     <chr> "Mean Cortisol levels", "Mean Cortisol levels", "Mean…
$ outcome_category <chr> "Hormones", "Hormones", "Hormones", "Hormones", "Horm…
$ method           <chr> "salivary samples, chewing on cotton swabs; cortisol …
$ units            <chr> "μg/dL", "μg/dL", "μg/dL", "μg/dL", "μg/dL", "μg/dL",…
$ timepoint_label  <chr> "Baseline before TSST (Pre)", "Baseline before TSST (…
$ timepoint_days   <chr> "Day 0 (baseline)", "Day 0 (baseline)", "Day 0 (basel…
$ group            <chr> "Intervention", "Control", "Intervention", "Control",…
$ analysis_set     <chr> "ITT", "ITT", "ITT", "ITT", "ITT", "ITT", "PP", "PP",…
$ n                <chr> "43.0", "41.0", "43.0", "41.0", "43.0", "41.0", "37.0…
$ mean             <chr> "0.224060151", "0.183236205", "0.223327067", "0.20182…
$ sd               <chr> "0.09335779", "0.047864605", "0.072890517", "0.051824…
$ se              

In [23]:
# Helper
num_from_char <- function(x) {
  if (is.numeric(x)) return(x)
  x_clean <- trimws(as.character(x))
  x_clean[x_clean %in% c("na", "NA", "n/a", "nr", "NR", "")] <- NA_character_
  suppressWarnings(as.numeric(x_clean))
}

In [24]:
# Harmonise `nonmental`

nonmental <- nonmental %>%
  mutate(
    # study_id numeric + factor
    study_id        = as.integer(study_id),
    study_id_factor = factor(study_id),

    # categorical/factor variables
    outcome_name     = factor(outcome_name),
    outcome_category = factor(outcome_category),
    units            = factor(units),
    timepoint_label  = factor(timepoint_label),
    timepoint_days   = factor(timepoint_days),
    group            = factor(group),
    analysis_set     = factor(analysis_set),

    # numeric variables
    n           = num_from_char(n),
    mean        = num_from_char(mean),
    sd          = num_from_char(sd),
    se          = num_from_char(se),
    ci_l        = num_from_char(ci_l),
    ci_u        = num_from_char(ci_u),
    median      = num_from_char(median),
    iqr_l       = num_from_char(iqr_l),
    iqr_u       = num_from_char(iqr_u),
    change_mean = num_from_char(change_mean),
    change_sd   = num_from_char(change_sd)
  ) %>%
  
  # drop unneeded columns
  dplyr::select(
    -method,
    -test_statistic,
    -p_value,
    -missing_data,
    -notes_on_values,
    -notes_on_rest
  )

In [25]:
# Harmonise 'timepoint_label'

harm_nonmental_timepoint_label <- function(x) {
  s <- str_to_lower(str_trim(as.character(x)))

  case_when(
    is.na(x) ~ NA_character_,

    # follow-ups
    str_detect(s, "follow") ~ "Follow-Up",

    # immediate post-immersion
    str_detect(s, "post-immersion") ~ "Post-Immersion",

    # during immersion (not explicitly post)
    str_detect(s, "during immersion") |
      (str_detect(s, "immersion") &
         !str_detect(s, "post") &
         !str_detect(s, "pre")) ~ "During-Immersion",

    # pre / baseline / autumn-pre / day 1 (treated as baseline)
    str_detect(s, "autumn \\(pre\\)") |
      str_detect(s, "\\bpre\\b") |
      str_detect(s, "day 1") ~ "Pre",

    # everything else: post
    TRUE ~ "Post"
  )
}

nonmental <- nonmental %>%
  mutate(
    timepoint_label_harmonised = factor(
      harm_nonmental_timepoint_label(timepoint_label),
      levels = c("Pre", "Post", "Post-Immersion", "During-Immersion", "Follow-Up")
    )
  )


In [26]:
# Harmonise 'timepoint_days' -> numeric days
tp_days_numeric_nonmental <- function(x) {
  s <- str_to_lower(str_trim(as.character(x)))
  s[is.na(s)] <- ""   # avoid NA in string operations

  out <- rep(NA_real_, length(s))

  # treat "nr", "na", etc. as NA
  is_missing <- s %in% c("nr", "na", "n/a", "")
  out[is_missing] <- NA_real_

  # pure integers -> days
  is_num <- str_detect(s, "^\\d+$")
  out[is_num] <- as.numeric(s[is_num])

  # baseline / same day
  out[str_detect(s, "day 0")]                     <- 0
  out[str_detect(s, "baseline before immersion")] <- 0
  out[str_detect(s, "baseline")]                  <- 0
  out[str_detect(s, "same day")]                  <- 0

  # explicit days
  out[str_detect(s, "after day 21")]              <- 21

  # day 0–90 (approximate as 90: end of follow-up window)
  out[str_detect(s, "day 0-90")]                  <- 90

  # weeks / months
  out[str_detect(s, "end of week 1")]             <- 1 * 7
  out[str_detect(s, "end of week 2")]             <- 2 * 7
  out[str_detect(s, "end of week 3")]             <- 3 * 7
  out[str_detect(s, "end of week 4")]             <- 4 * 7

  out[str_detect(s, "8 weeks")]                   <- 8 * 7  # e.g. "056 (8 weeks)"

  # minute-based time points: convert to fractions of a day

  # your previous general rules:
  out[str_detect(s, "post-immersion \\(within 15min")] <- 15  / 60 / 24
  out[str_detect(s, "\\b1min\\b")]                     <- 1   / 60 / 24
  out[str_detect(s, "\\b15min\\b")]                    <- 15  / 60 / 24
  out[str_detect(s, "\\b30min\\b")]                    <- 30  / 60 / 24
  out[str_detect(s, "\\b180min\\b")]                   <- 180 / 60 / 24
  out[str_detect(s, "30-min post")]                    <- 30  / 60 / 24

  # handle "during1min" / "during15min" explicitly
  out[str_detect(s, "during1min")]  <- 1   / 60 / 24
  out[str_detect(s, "during15min")] <- 15  / 60 / 24

  out
}

nonmental <- nonmental %>%
  mutate(
    timepoint_days_num = tp_days_numeric_nonmental(timepoint_days)
  )

In [27]:
# Harmonise group -> Intervention / Control / SingleGroup 

harm_nonmental_group <- function(x) {
  s <- str_to_lower(str_trim(as.character(x)))

  case_when(
    is.na(x) ~ NA_character_,

    str_detect(s, "singlegroup") ~ "SingleGroup",

    # pure control (no 'intervention' in string)
    str_detect(s, "control") & !str_detect(s, "intervention") ~ "Control",

    # anything with "intervention" (including Intervention(30s) etc.)
    str_detect(s, "intervention") ~ "Intervention",

    TRUE ~ NA_character_
  )
}

nonmental <- nonmental %>%
  mutate(
    group_harmonised = factor(
      harm_nonmental_group(group),
      levels = c("Intervention", "Control", "SingleGroup")
    )
  )

## Export all data frames
[Back to Table of Content](#table-of-content)

In [28]:
# Ensure processed directory exists
OUT_DIR <- "processed"
if (!dir.exists(OUT_DIR)) dir.create(OUT_DIR, recursive = TRUE)

In [29]:
# One Excel file with three sheets
write_xlsx(
  list(
    metadata  = metadata,
    mental    = mental,
    nonmental = nonmental
  ),
  path = file.path(OUT_DIR, "CWE_metadata_mental_nonmental.xlsx")
)

## Harmonise data frames into SPSS format
[Back to Table of Content](#table-of-content)

In [30]:
## Helper to make safe names

make_safe_name <- function(x) {
  x %>%
    as.character() %>%
    str_replace_all("[^A-Za-z0-9]+", "_") %>%  # non-alnum -> _
    str_replace_all("_+", "_") %>%             # collapse multiple _
    str_replace("^_|_$", "")                   # trim leading/trailing _
}

In [31]:
## Encode time in a compact, dot-free way
## - integer days -> d0, d1, d30, d90, d210, ...
## - otherwise   -> minutes: m30, m180, ...
make_tp_code <- function(x) {
  case_when(
    is.na(x) ~ "NA",
    abs(x - round(x)) < 1e-8 ~ paste0("d", round(x)),
    TRUE ~ paste0("m", round(x * 24 * 60))
  )
}

In [32]:
## Final step: enforce SPSS variable-name rules
## - allowed chars: letters, digits, _ @ # $
## - first char: letter or @ # $
## - max length: 64
make_spss_safe_names <- function(nms, maxlen = 64) {
  out <- nms

  # replace illegal chars
  out <- gsub("[^A-Za-z0-9_@#$]", "_", out)

  # ensure first char is acceptable
  bad_start <- !grepl("^[A-Za-z@#$]", out)
  out[bad_start] <- paste0("V", out[bad_start])

  # truncate
  out <- substr(out, 1, maxlen)

  # ensure uniqueness
  out <- make.unique(out, sep = "_")

  out
}

In [33]:
## Take cleaned 'mental' and define unique cells
mental_for_spss <- mental %>%
  distinct(
    study_id,
    instrument,
    subscale,
    timepoint_label_harmonised,
    timepoint_days_num,
    group_raw,
    analysis_set,
    n, mean, sd, se, ci_l, ci_u,
    median, iqr_l, iqr_u,
    change_mean, change_sd,
    .keep_all = FALSE
  ) %>%
  mutate(
    inst_safe = make_safe_name(instrument),
    sub_safe  = make_safe_name(subscale),
    tp_lab    = make_safe_name(timepoint_label_harmonised),
    tp_days_safe = make_tp_code(timepoint_days_num),
    grp_safe  = make_safe_name(group_raw),
    as_safe   = make_safe_name(analysis_set),

    var_stub = paste(
      "m", inst_safe, sub_safe, tp_lab, tp_days_safe, grp_safe, as_safe,
      sep = "_"
    )
  )

## sanity check: no collisions per (study_id, var_stub)
mental_for_spss %>%
  count(study_id, var_stub) %>%
  filter(n > 1)
# should be 0 rows

## Pivot to wide: one row per study, many mental variables
mental_wide <- mental_for_spss %>%
  group_by(study_id, var_stub) %>%
  summarise(
    n           = first(n),
    mean        = first(mean),
    sd          = first(sd),
    se          = first(se),
    ci_l        = first(ci_l),
    ci_u        = first(ci_u),
    median      = first(median),
    iqr_l       = first(iqr_l),
    iqr_u       = first(iqr_u),
    change_mean = first(change_mean),
    change_sd   = first(change_sd),
    .groups = "drop"
  ) %>%
  pivot_wider(
    id_cols    = study_id,
    names_from = var_stub,
    values_from = c(
      n, mean, sd, se, ci_l, ci_u,
      median, iqr_l, iqr_u,
      change_mean, change_sd
    ),
    names_sep = "_"
  )


study_id,var_stub,n
<int>,<chr>,<int>
2,m_BSI_anxiety_Follow_Up_NA_Control_ITT,2
2,m_BSI_anxiety_Follow_Up_NA_Intervention_30s_ITT,2
2,m_BSI_anxiety_Follow_Up_NA_Intervention_60s_ITT,2
2,m_BSI_anxiety_Follow_Up_NA_Intervention_90s_ITT,2
2,m_SF_36_mental_component_score_MCS_Follow_Up_NA_Control_ITT,2
2,m_SF_36_mental_component_score_MCS_Follow_Up_NA_Intervention_30s_ITT,2
2,m_SF_36_mental_component_score_MCS_Follow_Up_NA_Intervention_60s_ITT,2
2,m_SF_36_mental_component_score_MCS_Follow_Up_NA_Intervention_90s_ITT,2
2,m_SF_36_physical_component_score_PCS_Follow_Up_NA_Control_ITT,2


In [34]:
## Prepare 'nonmental' for wide format
nonmental_for_spss <- nonmental %>%
  distinct(
    study_id,
    outcome_name,
    outcome_category,
    units,
    timepoint_label_harmonised,
    timepoint_days_num,
    group,
    analysis_set,
    n, mean, sd, se, ci_l, ci_u,
    median, iqr_l, iqr_u,
    change_mean, change_sd,
    .keep_all = FALSE
  ) %>%
  mutate(
    out_safe   = make_safe_name(outcome_name),
    cat_safe   = make_safe_name(outcome_category),
    units_safe = make_safe_name(units),
    tp_lab     = make_safe_name(timepoint_label_harmonised),
    tp_days_safe = make_tp_code(timepoint_days_num),
    grp_safe   = make_safe_name(group),
    as_safe    = make_safe_name(analysis_set),

    var_stub = paste(
      "nm", out_safe, cat_safe, units_safe,
      tp_lab, tp_days_safe, grp_safe, as_safe,
      sep = "_"
    )
  )

## sanity check: no collisions per (study_id, var_stub)
nonmental_for_spss %>%
  count(study_id, var_stub) %>%
  filter(n > 1)
# should be 0 rows

## Pivot to wide
nonmental_wide <- nonmental_for_spss %>%
  group_by(study_id, var_stub) %>%
  summarise(
    n           = first(n),
    mean        = first(mean),
    sd          = first(sd),
    se          = first(se),
    ci_l        = first(ci_l),
    ci_u        = first(ci_u),
    median      = first(median),
    iqr_l       = first(iqr_l),
    iqr_u       = first(iqr_u),
    change_mean = first(change_mean),
    change_sd   = first(change_sd),
    .groups = "drop"
  ) %>%
  pivot_wider(
    id_cols    = study_id,
    names_from = var_stub,
    values_from = c(
      n, mean, sd, se, ci_l, ci_u,
      median, iqr_l, iqr_u,
      change_mean, change_sd
    ),
    names_sep = "_"
  )


study_id,var_stub,n
<int>,<chr>,<int>
1,nm_Mean_Cortisol_levels_Hormones_g_dL_Post_d21_Control_PP,3
1,nm_Mean_Cortisol_levels_Hormones_g_dL_Post_d21_Intervention_PP,3
1,nm_Mean_Cortisol_levels_Hormones_g_dL_Pre_d0_Control_ITT,3
1,nm_Mean_Cortisol_levels_Hormones_g_dL_Pre_d0_Intervention_ITT,3
7,nm_5_HIAA_Serotonin_metabolite_ng_ml_Post_NA_Control_PP,2
7,nm_5_HIAA_Serotonin_metabolite_ng_ml_Post_NA_Intervention_PP,2
7,nm_Adrenaline_Catecholamines_nmol_L_Post_NA_Control_PP,2
7,nm_Adrenaline_Catecholamines_nmol_L_Post_NA_Intervention_PP,2
7,nm_Endorphin_Peptides_ng_ml_Post_NA_Control_PP,2


In [35]:
## Clean metadata a bit (drop factor copy of study_id if present)
metadata_spss_base <- metadata %>%
  select(-matches("^study_id_factor$"))

## Final 1-row-per-study table
study_level_spss <- metadata_spss_base %>%
  left_join(mental_wide,   by = "study_id") %>%
  left_join(nonmental_wide, by = "study_id")


In [36]:
# Export to Excel for SPSS import
writexl::write_xlsx(study_level_spss,
  "processed/CWE_data_frame_SPSS.xlsx"
)